In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [19]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch.nn.functional as F

# Clase para gestionar el dataset
class RatingsDataset(Dataset):
    def __init__(self, users, items, ratings=None):
        self.users = users
        self.items = items
        self.ratings = ratings

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        if self.ratings is not None:
            return self.users[idx], self.items[idx], self.ratings[idx]
        else:
            return self.users[idx], self.items[idx]

# Modelo de factorización matricial con regularización
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=25, dropout_rate=0.3):
        super(MatrixFactorization, self).__init__()
        
        # Embeddings con menor dimensionalidad
        self.user_embedding = nn.Embedding(num_users + 1, embedding_dim)
        self.item_embedding = nn.Embedding(num_items + 1, embedding_dim)
        
        # Capa completamente conectada más simple
        self.fc = nn.Linear(embedding_dim * 2, 16)
        self.dropout = nn.Dropout(dropout_rate)
        self.output = nn.Linear(16, 1)
        
        # Inicialización de pesos
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, user, item):
        # Obtener embeddings y aplicar dropout para regularización
        user_emb = self.dropout(self.user_embedding(user))
        item_emb = self.dropout(self.item_embedding(item))
        
        # Concatenar embeddings
        x = torch.cat([user_emb, item_emb], dim=1)
        
        # Capa densa con regularización
        x = F.leaky_relu(self.fc(x), negative_slope=0.1)
        x = self.dropout(x)
        x = self.output(x)
        
        return x.squeeze()

In [20]:
# Cargar datos
print("Cargando datos...")
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

print(f"Tamaño del conjunto de entrenamiento: {len(train_data)}")
print(f"Tamaño del conjunto de prueba: {len(test_data)}")

# Codificar usuarios e ítems eficientemente
print("Codificando usuarios e ítems...")
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

train_users = torch.tensor(user_encoder.fit_transform(train_data['user']), dtype=torch.long)
train_items = torch.tensor(item_encoder.fit_transform(train_data['item']), dtype=torch.long)
train_ratings = torch.tensor(train_data['rating'].values, dtype=torch.float32)

# Crear mapeo para usuarios e ítems
user_mapping = {label: index for index, label in enumerate(user_encoder.classes_)}
item_mapping = {label: index for index, label in enumerate(item_encoder.classes_)}

# Función para manejar valores no vistos en el conjunto de prueba
def encode_with_mapping(values, mapping):
    return [mapping.get(val, len(mapping)) for val in values]

# Preparar datos de prueba
test_ids = torch.tensor(test_data['ID'].values, dtype=torch.long)
test_users = torch.tensor(encode_with_mapping(test_data['user'], user_mapping), dtype=torch.long)
test_items = torch.tensor(encode_with_mapping(test_data['item'], item_mapping), dtype=torch.long)

# Crear datasets y dataloaders
print("Creando dataloaders...")
train_dataset = RatingsDataset(train_users, train_items, train_ratings)
test_dataset = RatingsDataset(test_users, test_items)

batch_size = 1024  # Ajustar según memoria disponible
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Crear y entrenar modelo
print("Configurando modelo...")
num_users = len(user_encoder.classes_)
num_items = len(item_encoder.classes_)
embedding_dim = 32  # Reducir dimensionalidad para ahorrar memoria

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

model = MatrixFactorization(num_users, num_items, embedding_dim).to(device)

# Configurar entrenamiento
criterion = nn.MSELoss()
weight_decay = 0.01  # Parámetro de regularización L2
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

Cargando datos...
Tamaño del conjunto de entrenamiento: 390351
Tamaño del conjunto de prueba: 43320
Codificando usuarios e ítems...
Creando dataloaders...
Configurando modelo...
Usando dispositivo: cuda


In [ ]:
# Entrenamiento
print("Iniciando entrenamiento...")
num_epochs = 100
early_stopping_patience = 10  # Paciencia para early stopping
best_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        
        # Forward
        optimizer.zero_grad()
        prediction = model(user, item)
        loss = criterion(prediction, rating)
        
        # Backward
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0
    else:
        patience_counter += 1
    if patience_counter >= early_stopping_patience:
        print(f"Early stopping en época {epoch+1}")
        break
    # Imprimir progreso
    print(f"Época {epoch+1}/{num_epochs}, Pérdida: {avg_loss:.4f}")

Iniciando entrenamiento...
Época 1/100, Pérdida: 34.6033
Época 2/100, Pérdida: 7.4611
Época 3/100, Pérdida: 5.1246
Época 4/100, Pérdida: 4.6920
Época 5/100, Pérdida: 4.6247
Época 6/100, Pérdida: 4.5707
Época 7/100, Pérdida: 4.4922
Época 8/100, Pérdida: 4.4114
Época 9/100, Pérdida: 4.3106
Época 10/100, Pérdida: 4.1784
Época 11/100, Pérdida: 4.0656
Época 12/100, Pérdida: 3.9673
Época 13/100, Pérdida: 3.8816
Época 14/100, Pérdida: 3.7894
Época 15/100, Pérdida: 3.7229
Época 16/100, Pérdida: 3.6461
Época 17/100, Pérdida: 3.5704
Época 18/100, Pérdida: 3.5035
Época 19/100, Pérdida: 3.4375
Época 20/100, Pérdida: 3.3478
Época 21/100, Pérdida: 3.2803
Época 22/100, Pérdida: 3.2096


In [ ]:
# Generar predicciones para el conjunto de prueba
def generate_test_predictions():
    model.eval()
    all_predictions = []

    # Conjuntos de usuarios e ítems conocidos en el conjunto de entrenamiento
    known_users = set(train['user_id'].unique())
    known_items = set(train['item_id'].unique())
    global_mean = train['rating'].mean()  # Media global de ratings
    with torch.no_grad():
        for user, item in test_loader:
            user, item = user.to(device), item.to(device)

            # Convertir a listas para comprobar existencia en el conjunto de entrenamiento
            user_list = user.cpu().numpy()
            item_list = item.cpu().numpy()

            predictions = []
            for u, i in zip(user_list, item_list):
                if u not in known_users or i not in known_items:
                    predictions.append(global_mean)  # Usar media global
                else:
                    pred = model(torch.tensor([u], device=device), torch.tensor([i], device=device))
                    predictions.append(torch.clamp(pred, min=1.0, max=10.0).item())

            all_predictions.extend(predictions)

    # Crear dataframe con predicciones
    results = pd.DataFrame({
        'ID': test_ids.numpy(),
        'rating': all_predictions
    })

    return results

# Evaluación con validación cruzada
def evaluate_model():
    # Usaremos una parte de los datos de entrenamiento como validación
    val_size = int(0.1 * len(train_dataset))
    train_subset, val_subset = torch.utils.data.random_split(
        train_dataset, [len(train_dataset) - val_size, val_size]
    )
    val_loader = DataLoader(val_subset, batch_size=batch_size)
    
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for user, item, rating in val_loader:
            user, item, rating = user.to(device), item.to(device), rating.to(device)
            pred = model(user, item)
            predictions.extend(pred.cpu().numpy())
            actuals.extend(rating.cpu().numpy())
    
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actuals, predictions)
    
    return mse, rmse, mae

In [ ]:

# Evaluación
print("Evaluando modelo...")
mse, rmse, mae = evaluate_model()
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

# Generar predicciones
print("Generando predicciones...")
predictions = generate_test_predictions()
print("Primeras 5 predicciones:")
print(predictions.head())

# Guardar predicciones
predictions.to_csv('predictions.csv', index=False)
print("Predicciones guardadas en 'predictions.csv'")

Evaluando modelo...
MSE: 1.2230
RMSE: 1.1059
MAE: 0.8395
Generando predicciones...
Primeras 5 predicciones:
   ID    rating
0   0  6.246171
1   1  8.081882
2   2  6.732179
3   3  8.502872
4   4  5.059886
Predicciones guardadas en 'predictions.csv'
